In [ ]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
import numpy as np
from PIL import Image
import csv
import zipfile
from google.colab import files

# CIFAR-10 Class Names
CIFAR10_CLASSES = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

# Define the device globally
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Log results to a CSV file
def log_results(topology, lr, accuracy, class_accuracies, output_file='results.csv'):
    with open(output_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([str(topology), lr, accuracy] + class_accuracies)

# Load full CIFAR-10 dataset
def load_cifar10_full(batch_size=128):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load full training and test sets
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    # Create DataLoaders
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    return trainloader, testloader

# Define a dynamic neural network model
class CIFAR10Model(nn.Module):
    def __init__(self, topology):
        super(CIFAR10Model, self).__init__()
        layers = []
        input_size = 3072  # CIFAR-10 flattened input size (32x32x3)
        for size in topology:
            layers.append(nn.Linear(input_size, size))
            layers.append(nn.ReLU())
            input_size = size
        layers.pop()  # Remove the last ReLU for the output layer
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        return self.network(x)

# Create folders to save true and false predictions
def create_output_folders(base_dir='output'):
    true_dir = os.path.join(base_dir, 'true')
    false_dir = os.path.join(base_dir, 'false')
    shutil.rmtree(base_dir, ignore_errors=True)  # Remove old folders if they exist
    os.makedirs(true_dir, exist_ok=True)
    os.makedirs(false_dir, exist_ok=True)
    return true_dir, false_dir

# Save misclassified and correctly classified images
def save_image(image, folder, filename):
    if not os.path.exists(folder):
        os.makedirs(folder)
    image = image.squeeze().permute(1, 2, 0).numpy()  # Convert to HWC format
    image = (image * 255).astype('uint8')  # Rescale to 0-255
    img = Image.fromarray(image)
    img.save(os.path.join(folder, filename))

# Train and evaluate the model
def train_and_evaluate_model(topology, learning_rate, epochs=50, batch_size=128, base_dir='output'):
    trainloader, testloader = load_cifar10_full(batch_size=batch_size)

    # Define the model and move it to the global device
    model = CIFAR10Model(topology).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scaler = GradScaler()  # For mixed precision training

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}")

    # Evaluation loop and saving images
    model.eval()
    true_dir, false_dir = create_output_folders(base_dir)
    correct = 0
    total = 0
    class_correct = [0] * 10  # To track correct predictions for each class
    class_total = [0] * 10    # To track total images for each class
    with torch.no_grad():
        for i, (images, labels) in enumerate(testloader):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Calculate class-wise accuracy
            for idx in range(len(images)):
                label = labels[idx]
                is_correct = (predicted[idx] == label).item()
                class_total[label] += 1
                class_correct[label] += is_correct

                # Save images
                folder = true_dir if is_correct else false_dir
                filename = f'image_{i * len(images) + idx}.png'
                save_image(images[idx].cpu(), folder, filename)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

    # Class-wise accuracy
    class_accuracies = []
    for i in range(10):
        class_accuracy = 100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0
        print(f"Accuracy of {CIFAR10_CLASSES[i]}: {class_accuracy:.2f}%")
        class_accuracies.append(class_accuracy)

    print(f"Saved images to '{base_dir}/true' (correct) and '{base_dir}/false' (incorrect')")

    return accuracy, class_accuracies

# Zip the output folder for downloading
def zip_output_folder(folder_name='output', zip_name='output.zip'):
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for root, dirs, files in os.walk(folder_name):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_name))
    print(f"Zipped folder '{folder_name}' into '{zip_name}'")

# Main training and logging
with open('results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Topology', 'Learning Rate', 'Accuracy', 'Plane Accuracy', 'Car Accuracy', 'Bird Accuracy',
                     'Cat Accuracy', 'Deer Accuracy', 'Dog Accuracy', 'Frog Accuracy', 'Horse Accuracy', 'Ship Accuracy', 'Truck Accuracy'])

topologies = [
    [1024, 128, 10],
    [1024, 128, 256, 10],
    [1024, 128, 256, 256, 10],
    [1024, 128, 256, 256, 128, 10],
    [1024, 128, 256, 512, 256, 10],
    [1024, 128, 256, 512, 256, 128, 10],
    [1024, 128, 256, 512, 1024, 512, 256, 10],
    [1024, 128, 256, 512, 1024, 512, 256, 128, 10],
    [1024, 128, 256, 512, 1024, 2048, 1024, 512, 256, 128, 10],
    [1024, 128, 256, 512, 1024, 512, 1024, 256, 128, 10]
]

learning_rates = [0.001, 0.005]

for lr in learning_rates:
    for topology in topologies:
        print(f"Training topology: {topology} with learning rate: {lr}")
        accuracy, class_accuracies = train_and_evaluate_model(topology, learning_rate=lr, epochs=50, base_dir='output')
        log_results(topology, lr, accuracy, class_accuracies)

# Zip the output folder after training
zip_output_folder()

# If using Google Colab, provide a download link
files.download('output.zip')


Using device: cuda
Training topology: [1024, 128, 10] with learning rate: 0.001


100%|██████████| 170M/170M [00:13<00:00, 12.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


<ipython-input-1-8b3625ba0b14>:90: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # For mixed precision training
<ipython-input-1-8b3625ba0b14>:100: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50, Loss: 1.6333
Epoch 2/50, Loss: 1.4224
Epoch 3/50, Loss: 1.3063
Epoch 4/50, Loss: 1.2222
Epoch 5/50, Loss: 1.1378
Epoch 6/50, Loss: 1.0588
Epoch 7/50, Loss: 0.9858
Epoch 8/50, Loss: 0.9109
Epoch 9/50, Loss: 0.8394
Epoch 10/50, Loss: 0.7711
Epoch 11/50, Loss: 0.7081
Epoch 12/50, Loss: 0.6444
Epoch 13/50, Loss: 0.5896
Epoch 14/50, Loss: 0.5370
Epoch 15/50, Loss: 0.4991
Epoch 16/50, Loss: 0.4570
Epoch 17/50, Loss: 0.4292
Epoch 18/50, Loss: 0.3864
Epoch 19/50, Loss: 0.3755
Epoch 20/50, Loss: 0.3507
Epoch 21/50, Loss: 0.3362
Epoch 22/50, Loss: 0.3087
Epoch 23/50, Loss: 0.2856
Epoch 24/50, Loss: 0.2707
Epoch 25/50, Loss: 0.2687
Epoch 26/50, Loss: 0.2548
Epoch 27/50, Loss: 0.2540
Epoch 28/50, Loss: 0.2263
Epoch 29/50, Loss: 0.2394
Epoch 30/50, Loss: 0.2267
Epoch 31/50, Loss: 0.2107
Epoch 32/50, Loss: 0.2018
Epoch 33/50, Loss: 0.2043
Epoch 34/50, Loss: 0.2002
Epoch 35/50, Loss: 0.1947
Epoch 36/50, Loss: 0.1863
Epoch 37/50, Loss: 0.1893
Epoch 38/50, Loss: 0.1827
Epoch 39/50, Loss: 0.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>